In [68]:
# Dependencies for entire notebook here
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [69]:
# Functions for creating weights and biases
# https://www.tensorflow.org/get_started/mnist/pros
def weight_variable(shape, name=None):    
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name=None):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

# Functions for convolution and pooling functions
def conv2d(x, W, name=None):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME', name=name)

def max_pooling_2x2(x, name=None):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name=name)


In [70]:
def load_saved_adversarial():    
    local_path = "C:\\Users\\alonh\Documents\\Thesis\\MNIST-adversarial-images\\"
    original_number = np.load(local_path + "original_number.npy")
    target_number = np.load(local_path + "target_number.npy")
    prob_list = np.load(local_path + "prob_list.npy")
    adv_img_list = np.load(local_path + "adv_img_list.npy")
    return original_number, target_number, prob_list, adv_img_list


Extracting /tmp/data/train-images-idx3-ubyte.gz


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [71]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [72]:
#load_adversarial
original_number, target_number, prob_list, adv_img_list = load_saved_adversarial()


In [73]:
# uniting and shuffeling between adv inputs and benign
# creating labels as 2d array which [0,1] is adv
number_of_adv_images = adv_img_list.shape[0]
adv_label = np.zeros(number_of_adv_images)
adv_label = np.c_[adv_label, np.ones(number_of_adv_images)]

number_of_benign = 1000
benign_label = np.ones(number_of_benign)
benign_label = np.c_[benign_label, np.zeros(number_of_benign)]
# uniting adv and benign
united_labels = np.concatenate((adv_label, benign_label), axis=0)
united_images = np.concatenate((adv_img_list, mnist.train.images[0:number_of_benign]), axis=0)
united_images_and_labels = np.c_[united_images, united_labels]
np.random.shuffle(united_images_and_labels)
#add precondition
images_data, labels_data = np.hsplit(united_images_and_labels, [united_images_and_labels.shape[1]-2])
train_test_rate = 0.7
train_size = int(images_data.shape[0]*train_test_rate)
image_train_data, image_test_data = np.split(images_data, [train_size])
label_train_data, label_test_data = np.split(labels_data, [train_size])


In [74]:
# should start run from here
tf.reset_default_graph()
sess = tf.InteractiveSession()
W_conv1 = tf.get_variable("W_conv1", shape=[5, 5, 1, 32])
b_conv1 = tf.get_variable("b_conv1", shape=[32])
W_conv2 = tf.get_variable("W_conv2", shape=[5, 5, 32, 64])
b_conv2 = tf.get_variable("b_conv2", shape=[64])
saver = tf.train.Saver(var_list={'W_conv1': W_conv1, 'b_conv1': b_conv1, 'W_conv2': W_conv2, 'b_conv2': b_conv2})
saver.restore(sess, "C:\\Users\\alonh\Documents\\Thesis\\MNIST-adversarial-images\\original-nn-data.ckpt")


ValueError: Variable W_conv1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "c:\users\alonh\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\ops.py", line 1717, in __init__
    self._traceback = tf_stack.extract_stack()
  File "c:\users\alonh\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\ops.py", line 3155, in create_op
    op_def=op_def)
  File "c:\users\alonh\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\util\deprecation.py", line 454, in new_func
    return func(*args, **kwargs)


In [75]:
# Create placeholders nodes for images and label inputs
# Create placeholders nodes for images and label inputs
x = tf.placeholder(tf.float32, shape=[None, 784], name='xDetector')
y_ = tf.placeholder(tf.float32, shape=[None, 2], name='yDetector')


In [76]:
# modified nn based on original
to_initialize = []
# Input layer
x_image_new = tf.reshape(x, [-1,28,28,1]) # mnist image comes in as 784 vector

# # Conv layer 1 - 32x5x5 - using same weights and biases as the pretrained
# W_conv1 = weight_variable([5, 5, 1, 32])
# b_conv1 = bias_variable([32])
x_conv1 = tf.nn.relu(conv2d(x_image_new, W_conv1) + b_conv1, name="x_conv1_new")
x_pool1 = max_pooling_2x2(x_conv1, name="x_pool1_new")
# Conv detector layer 1  - 32x5x5
W_conv1_detector = weight_variable([5, 5, 32, 32], name="W_conv1_detector")
b_conv1_detector = bias_variable([32], name="b_conv1_detector")
x_conv1_detector = tf.nn.relu(conv2d(x_pool1, W_conv1_detector) + b_conv1_detector, name="x_conv1_detector")
x_pool1_detector = max_pooling_2x2(x_conv1_detector, name="x_pool1_detector")

# # Conv layer 2 - 64x5x5
# W_conv2 = weight_variable([5, 5, 32, 64])
# b_conv2 = bias_variable([64])
x_conv2 = tf.nn.relu(conv2d(x_pool1, W_conv2) + b_conv2, name="x_conv2_new")
x_pool2 = max_pooling_2x2(x_conv2, name="x_pool2_new")

# stacking conv2 output and x_pool1 detector output
conv_detector_2_input = tf.concat([x_conv2, x_pool1], 3)


# Conv detector layer 2 - 64x5x5
W_conv2_detector = weight_variable([5, 5, 96, 64], name="W_conv2_detector")
b_conv2_detector = bias_variable([64], name="b_conv2_detector")
x_conv2_detector = tf.nn.relu(conv2d(conv_detector_2_input, W_conv2_detector) + b_conv2_detector, name="x_conv2_detector")
x_pool2_detector = max_pooling_2x2(x_conv2_detector, name="x_pool2_detector")
# Flatten - keras 'flatten'
pool_shape = x_pool2_detector.shape
shape_size = (pool_shape[1]*pool_shape[2]*pool_shape[3]).value
x_flat_detector = tf.reshape(x_pool2_detector, [-1, shape_size], name="x_flat_detector")


W_fc1_detector = weight_variable([shape_size, 1024], name="W_fc1_detector") # max pooling reduced image to 7x7
b_fc1_detector = bias_variable([1024], name="b_fc1_detector")
x_fc1_detector = tf.nn.relu(tf.matmul(x_flat_detector, W_fc1_detector) + b_fc1_detector, name="x_fc1_detector")

# Regularization with dropout
keep_prob_detector = tf.placeholder(tf.float32, name="keep_prob_detector")
x_fc1_drop_detector = tf.nn.dropout(x_fc1_detector, keep_prob_detector, name="x_fc1_drop_detector")


# Classification layer
W_fc2_detector = weight_variable([1024, 2], name="W_fc2_detector")
b_fc2_detector = bias_variable([2], name="b_fc2_detector")
y_conv= tf.matmul(x_fc1_drop_detector, W_fc2_detector) + b_fc2_detector


In [77]:
# Probabilities - output from model (not the same as logits)
# CHANGE!!-----------------------------------------------------------------
y = tf.nn.softmax(y_conv, name="y_new")

In [78]:
# Loss and optimizer
cross_entropy2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv), name="cross_entropy2")
# add variables to optimize here:
var_list = [W_conv1_detector, W_conv2_detector, W_fc2_detector, b_conv1_detector, b_conv2_detector, b_fc2_detector]
train_step2 = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy2, var_list=var_list, name="train_step2")

In [79]:
# Setup to test accuracy of model
correct_prediction2 = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1), name="correct_prediction2")
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32), name="accuracy2")


In [80]:
def initialize_uninitialized_global_variables(sess):
    """
    Only initializes the variables of a TensorFlow session that were not
    already initialized.
    :param sess: the TensorFlow session
    :return:
    """
    # List all global variables
    global_vars = tf.global_variables()

    # Find initialized status for all variables
    is_var_init = [tf.is_variable_initialized(var) for var in global_vars]
    is_initialized = sess.run(is_var_init)

    # List all variables that were not initialized previously
    not_initialized_vars = [var for (var, init) in
                            zip(global_vars, is_initialized) if not init]
    # for uninit in not_initialized_vars:
    #     print(uninit)
    # Initialize all uninitialized variables found, if any
    if len(not_initialized_vars):        
        sess.run(tf.variables_initializer(not_initialized_vars)) 

initialize_uninitialized_global_variables(sess)


In [81]:
# Train model
# Run once to get the model to a good confidence level
batch_size = 100
batch_index = 0
image_train_size = image_train_data.shape[0]
number_of_steps = image_train_size / batch_size
for i in range(int(number_of_steps)):
    # if(batch_index>=image_train_data.shape[0]):
    #     print("reached limit of train data current index: ", batch_index, "data size: ", image_train_data.shape[0])
    #     break
    batch_image = image_train_data[batch_index: (batch_index + batch_size)]
    batch_label = label_train_data[batch_index: (batch_index + batch_size)]
    if i % 2 == 0:
        train_accuracy = accuracy2.eval(session=sess, feed_dict={x: batch_image, y_:batch_label, keep_prob_detector: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    train_step2.run(feed_dict={x: batch_image, y_: batch_label, keep_prob_detector: 0.4})
    batch_index = batch_index + batch_size


step 0, training accuracy 0.56
step 2, training accuracy 0.5
step 4, training accuracy 0.45


step 6, training accuracy 0.48
step 8, training accuracy 0.49
step 10, training accuracy 0.56
step 12, training accuracy 0.49


In [82]:
# Run trained model against test data
print("test accuracy %g"%accuracy2.eval(session=sess, feed_dict={x: image_test_data, 
                                                  y_: label_test_data, keep_prob_detector: 1.0}))

test accuracy 0.517544


In [83]:
sess.close()